In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv("CKD.csv")

In [3]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [4]:
indep=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]

dep=dataset[['classification_yes']]

In [5]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=1/3,random_state=0)

In [6]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [7]:
from sklearn.ensemble import RandomForestClassifier

In [8]:
from sklearn.model_selection import GridSearchCV

param_grid={'criterion':['gini','entropy'],
            'max_features':['auto','sqrt','log2'],
            'n_estimators':[10,100]}
grid=GridSearchCV(RandomForestClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')

#fiting the model for grid search
grid.fit(x_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             scoring='f1_weighted', verbose=3)

In [9]:
#print the best parameter aftertunning
#print (grid.best_params_)
re=grid.cv_results_
grid_prediction=grid.predict(x_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_prediction)

#print classification Report
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_prediction)

In [10]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_prediction,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'gini', 'max_features': 'auto', 'n_estimators': 100}: 0.9924946382275899


In [11]:
print("The confusion matrix:\n",cm)
print("The report:\n",clf_report)

The confusion matrix:
 [[51  0]
 [ 1 81]]
The report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99        51
           1       1.00      0.99      0.99        82

    accuracy                           0.99       133
   macro avg       0.99      0.99      0.99       133
weighted avg       0.99      0.99      0.99       133



In [12]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

1.0

In [13]:
table=pd.DataFrame.from_dict(re)
table


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.059058,0.023256,0.012145,0.004029,gini,auto,10,"{'criterion': 'gini', 'max_features': 'auto', ...",1.000000,0.962264,0.962573,1.000000,0.981031,0.981174,0.016807,3
1,0.312185,0.016983,0.038152,0.013556,gini,auto,100,"{'criterion': 'gini', 'max_features': 'auto', ...",1.000000,0.961755,0.981217,0.981031,1.000000,0.984801,0.014284,1
2,0.046184,0.007923,0.009709,0.009684,gini,sqrt,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.890467,0.943041,0.962573,0.981031,0.943093,0.944041,0.030278,12
3,0.268025,0.028866,0.041703,0.008363,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.981569,0.961755,0.981217,0.981031,0.981031,0.977321,0.007785,7
4,0.044131,0.013899,0.006486,0.004504,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",0.981569,0.905069,0.944023,0.962264,0.981217,0.954829,0.028494,11
5,0.288733,0.034496,0.029146,0.006694,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.981569,0.961755,0.981217,0.981031,0.981217,0.977358,0.007803,5
6,0.038008,0.006437,0.006890,0.003955,entropy,auto,10,"{'criterion': 'entropy', 'max_features': 'auto...",0.981569,0.981014,0.944023,0.981031,0.981031,0.973734,0.014857,8
7,0.286884,0.039813,0.030765,0.010711,entropy,auto,100,"{'criterion': 'entropy', 'max_features': 'auto...",1.000000,0.961755,0.981217,0.981031,1.000000,0.984801,0.014284,1
8,0.040489,0.000025,0.008880,0.001003,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.963284,0.961755,1.000000,0.961826,1.000000,0.977373,0.018483,4
9,0.301225,0.016495,0.028761,0.006678,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.981569,0.942166,0.981217,0.981031,0.981031,0.973403,0.015620,9


In [14]:
import pickle                                    #used to save model
filename="finalized_model_Random_Forest.sav"  
pickle.dump(grid,open(filename,'wb'))

In [15]:
model=pickle.load(open(filename,'rb'))

In [17]:
age_input=int(input("age:"))
bp_input=float(input("bp:"))
su_input=float(input("su:"))
bp_input=float(input('bp:'))
al_input=float(input("al:"))
bgr_input=float(input("bgr"))
bu_input=float(input("bu:"))
sc_input=float(input("sc"))
sod_input=float(input("sod:"))
pot_input=float(input("pot:"))
hrmo_input=float(input("hrmo:"))
pc_input=int(input("pc_normal:"))
pcc_input=int(input("pcc_present:"))
ba_input=int(input("ba_present:"))
cad_input=int(input("cad_yes_no:"))
htn_input=int(input("htn_yes_no:"))
dm_input=int(input("dm_yes_no:"))
appet_input=int(input("appet_yes_no:"))
ane_input=int(input("ane_yes_no:"))
pe_input=int(input("pe:"))


age:23
bp:56.90
su:8.90
bp:6.975
al:7.094
bgr0.987
bu:3.974
sc7.80
sod:5.90
pot:8.40
hrmo:5.0
pc_normal:0
pcc_present:0
ba_present:1
cad_yes_no:1
htn_yes_no:0
dm_yes_no:1
appet_yes_no:0
ane_yes_no:1
pe:0


In [18]:
Future_prediction=model.predict([[age_input,bp_input,su_input,al_input,bgr_input,bu_input,sc_input,sod_input,pot_input,
                                 hrmo_input,pc_input,pcc_input,ba_input,cad_input,htn_input,
                                 dm_input,appet_input,ane_input,pe_input]])
print("Future_prediction={}".format(Future_prediction))

Future_prediction=[1]
